Task: implement versions of the naive exact matching and Boyer-Moore algorithms that additionally count and return (a) the number of character comparisons performed and (b) the number of alignments tried.

## Naive exact matching

In [1]:
def naive_with_counts(p, t):
    occurrences = []
    num_align = 0
    num_char  = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        num_align += 1
        match = True
        for j in range(len(p)):  # loop over characters
            num_char += 1
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences, num_align, num_char

### Examples

In [2]:
p = 'word'
t = 'there would have been a time for such a word' # 3 comparisons at "would", third to reject
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 41 46


In [3]:
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 20 35


## Boyer-Moore

In [4]:
from bm_preproc import BoyerMoore

def boyer_moore_with_counts(p, p_bm, t):
    """ Do Boyer-Moore matching """
    i = 0
    occurrences = []
    num_align = 0
    num_char  = 0
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        num_align += 1
        for j in range(len(p)-1, -1, -1):
            num_char += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, num_align, num_char

### Examples

In [5]:
t = 'GCTAGCTCTACGAGTCTA'
p = 'TCTA'
p_bm = BoyerMoore(p, alphabet='ACGT')
boyer_moore_with_counts(p, p_bm, t)

([6, 14], 5, 14)

In [6]:
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 12 15


In [7]:
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 5 18


## Running on real data

```
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
```

In [8]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [9]:
p = 'ATTA'
p_bm = BoyerMoore(p, alphabet='ACGT')
chr1_genome = readGenome("chr1.GRCh38.excerpt.fasta")

In [10]:
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, chr1_genome)
print(len(occurrences), num_alignments, num_character_comparisons)

6361 799997 1154588


In [11]:
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, chr1_genome)
print(len(occurrences), num_alignments, num_character_comparisons)

6361 297247 445445


### Answers

In [12]:
# question 1 to 3
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
p_bm = BoyerMoore(p, alphabet='ACGT')

occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, chr1_genome)
print("naive: ", num_alignments, num_character_comparisons)

occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, chr1_genome)
print("boyer-moore: ", num_alignments, num_character_comparisons)

naive:  799954 984143
boyer-moore:  127974 165191


# Question 4 & 5

We are implementing approximate matching assisted with k-mer indices. For a pattern of length 24, we want to allow at most 2 mismatches. Using pigeonhole principle, $24 / (2 + 1) = 8$, so an 8-mer index is appropriate.

In [13]:
from kmer_index import Index

def queryIndex(p, t, index):
    k = index.k
    offsets = []
    for i in index.query(p):
        if p[k:] == t[i+k:i+len(p)]:  # verify that rest of P matches
            offsets.append(i)
    return offsets

In [14]:
t = 'ACTTGGAGATCTTTGAGGCTAGGTATTCGGGATCGAAGCTCATTTCGGGGATCGATTACGATATGGTGGGTATTCGGGA'
p = 'GGTATTCGGGA'
index = Index(t, 8)  # builds an 8-mer index
print(queryIndex(p, t, index))

[21, 68]


In [15]:
# constuct 8-mer index for chromosome 1
index_chr1 = Index(chr1_genome, 8)
index_chr1.index[:5]

[('AAAAAAAA', 1867),
 ('AAAAAAAA', 1868),
 ('AAAAAAAA', 2631),
 ('AAAAAAAA', 2995),
 ('AAAAAAAA', 2996)]

In [16]:
def approximate_match_2mm_index(p, t, t_index, n = 2):
    assert len(p) == 24, "pattern must be length 24"
    segment_length = int(round(len(p) / (n+1)))
    assert segment_length == 8
    all_matches = set()
    index_hits = set()
    for i in range(n+1): # three segments
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        matches = queryIndex(p[start:end], t, t_index)
        # Extend matching segments to see if whole p matches
        for m in matches:
            index_hits.add(m)
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches), list(index_hits)

In [89]:
occurrences, index_hits = approximate_match_2mm_index("GGCGCGGTGGCTCACGCCTGTAAT", chr1_genome, index_chr1)
print(len(occurrences))
print(occurrences)

19
[84641, 160162, 724927, 273669, 147558, 364263, 421221, 681737, 717706, 465647, 429299, 747359, 657496, 160729, 56922, 635931, 191452, 551134, 262042]


In [18]:
# compare to naive 2mm, since it should give same answer
def naive_2mm(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = 0
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                match += 1
        if match <= 2:
            occurrences.append(i)  # all chars matched; record
    return occurrences

occurrences_naive = naive_2mm("GGCGCGGTGGCTCACGCCTGTAAT", chr1_genome)
print(len(occurrences_naive))
print(occurrences_naive)

19
[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]


In [19]:
print(len(index_hits))

# compare index hits by matching the three segments separately
p = "GGCGCGGTGGCTCACGCCTGTAAT"
len(naive_with_counts(p[0:8], chr1_genome)[0]) + len(naive_with_counts(p[8:16], chr1_genome)[0]) + len(naive_with_counts(p[16:24], chr1_genome)[0])

90


90

## Question 6

In [20]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [31]:
# produced 3-mers, using every other 2nd letter
ind = SubseqIndex('ATATAT', k = 3, ival = 2)
print(ind.index)

[('AAA', 0), ('TTT', 1)]


In [32]:
ind.query('TTATAT')

[]

In [33]:
ind.query('TATAT') # TTT is in

[1]

Task: implement an approximate matching function that uses subsequence indexes, and counts the number of index hits.

In [34]:
def querySubseqIndex(p, t, index):
    k = index.k
    offsets = []
    for i in index.query(p):
        if p == t[i:i+len(p)]:
            offsets.append(i)
    return offsets

In [79]:
def query_subseq(p, t, t_index, n = 2):
    assert len(p) == 24, "pattern must be length 24"
    all_matches = set()
    index_hits = set()
    for i in range(n+1): # three different offsets
        start = 0
        end = len(p)
        matches = querySubseqIndex(p[i:], t, t_index)
        # Extend matching segments to see if whole p matches
        for m in matches:
            index_hits.add(m)
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start - i) # to account for the offset
    return list(all_matches), list(index_hits)

In [73]:
# test example 1
t = 'to-morrow and to-morrow and to-morrow creeps in this petty pace'
p = 'to-morrow and to-morrow '
subseq_ind = SubseqIndex(t, 8, 3)
query_subseq(p, t, subseq_ind)

[0, 14]
[0, 14]
[0, 14]


([0, 14], [0, 1, 2, 14, 15, 16])

In [75]:
# test example 2
# !wget http://www.gutenberg.org/ebooks/1110.txt.utf-8

t = open('1110.txt.utf-8').read()
p = 'English measure backward'
subseq_ind = SubseqIndex(t, 8, 3)
query_subseq(p, t, subseq_ind)

[132574]
[132574]
[132574]


([132574], [132576, 132574, 132575])

In [96]:
subseq_ind_chr1 = SubseqIndex(chr1_genome, 8, 3)
occurrences_subseq, index_hits_subseq = query_subseq("GGCGCGGTGGCTCACGCCTGTAAT", chr1_genome, subseq_ind_chr1)

In [103]:
occurrences_subseq.sort()
print(occurrences_subseq)

[56922, 262042, 364263, 429299, 657496, 717706]


In [98]:
len(index_hits_subseq)

17

In [102]:
# doesn't seem to add up though
occurrences, index_hits = approximate_match_2mm_index("GGCGCGGTGGCTCACGCCTGTAAT", chr1_genome, index_chr1)
occurrences.sort()
print(occurrences)

[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]
